# Deep and Wide Network Approach

Many diseases, including cancer, are believed to have a contributing factor in common. Ion channels are pore-forming proteins present in animals and plants. They encode learning and memory, help fight infections, enable pain signals, and stimulate muscle contraction. If scientists could better study ion channels, which may be possible with the aid of machine learning, it could have a far-reaching impact.

When ion channels open, they pass electric currents. Existing methods of detecting these state changes are slow and laborious. Humans must supervise the analysis, which imparts considerable bias, in addition to being tedious. These difficulties limit the volume of ion channel current analysis that can be used in research. Scientists hope that technology could enable rapid automatic detection of ion channel current events in raw data.

Technology to analyze electrical data in cells has not changed significantly over the past 20 years. If we better understand ion channel activity, the research could impact many areas related to cell health and migration. From human diseases to how climate change affects plants, faster detection of ion channels could greatly accelerate solutions to major world problems.

<img src="resources/channels.jpg">

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. Neural Nets can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

from kaggle_secrets import UserSecretsClient

seed = 42
        
import numpy as np
import pandas as pd
# to make this notebook's output stable across runs
np.random.seed(seed)
tf.random.set_seed(seed)

# Common imports
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import numpy as np 
import warnings
import gc
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

In [ ]:
def read_data():
    if os.path.isdir('/kaggle/input/'):
        path_str = '/kaggle/input/'
    else:
        path_str = '/input/'
    '''Read the data'''
    print('Reading training, testing and submission data...')
    train = pd.read_csv(path_str+'liverpool-ion-switching/train.csv')
    test = pd.read_csv(path_str+'liverpool-ion-switching/test.csv')
    submission = pd.read_csv(path_str+'liverpool-ion-switching/sample_submission.csv', dtype={'time':str})
    print('Train set has {} rows and {} columns'.format(train.shape[0], train.shape[1]))
    print('Test set has {} rows and {} columns'.format(test.shape[0], test.shape[1]))
    return train, test, submission

train, test, submission = read_data()

From the exploratory data analysis we know that we have 10 batches for training and 4 batches in the test. The batches are independent from each other so our job is to extract features in those batches to improve the network performance

In [ ]:
# concatenate data
batch = 50
total_batches = 14
train['set'] = 'train'
test['set'] = 'test'
data = pd.concat([train, test])
for i in range(int(total_batches)):
    data.loc[(data['time'] > i * batch) & (data['time'] <= (i + 1) * batch), 'batch'] = i + 1
train = data[data['set'] == 'train']
test = data[data['set'] == 'test']
del data

In [3]:
def preprocess(train, test):
    ''' Data preprocessing to extract mean, min, max and std''' 
    pre_train = train.copy()
    pre_test = test.copy()
    
    batch1 = pre_train[pre_train["batch"] == 1]
    batch2 = pre_train[pre_train["batch"] == 2]
    batch3 = pre_train[pre_train["batch"] == 3]
    batch4 = pre_train[pre_train["batch"] == 4]
    batch5 = pre_train[pre_train["batch"] == 5]
    batch6 = pre_train[pre_train["batch"] == 6]
    batch7 = pre_train[pre_train["batch"] == 7]
    batch8 = pre_train[pre_train["batch"] == 8]
    batch9 = pre_train[pre_train["batch"] == 9]
    batch10 = pre_train[pre_train["batch"] == 10]
    batch11 = pre_test[pre_test['batch'] == 11]
    batch12 = pre_test[pre_test['batch'] == 12]
    batch13 = pre_test[pre_test['batch'] == 13]
    batch14 = pre_test[pre_test['batch'] == 14]
    batches = [batch1, batch2, batch3, batch4, batch5, batch6, batch7, batch8, batch9, batch10, batch11, batch12, batch13, batch14]
    
    for batch in batches:
        for feature in ['signal']:
            # some random rolling features
            for window in [100, 1000, 10000]:
                # roll backwards
                batch[feature + 'mean_t' + str(window)] = batch[feature].shift(1).rolling(window).mean()
                batch[feature + 'std_t' + str(window)] = batch[feature].shift(1).rolling(window).std()
                batch[feature + 'min_t' + str(window)] = batch[feature].shift(1).rolling(window).min()
                batch[feature + 'max_t' + str(window)] = batch[feature].shift(1).rolling(window).max()
                min_max = (batch[feature] - batch[feature + 'min_t' + str(window)]) / (batch[feature + 'max_t' + str(window)] - batch[feature + 'min_t' + str(window)])
                batch['norm_t' + str(window)] = min_max * (np.floor(batch[feature + 'max_t' + str(window)]) - np.ceil(batch[feature + 'min_t' + str(window)]))
                
    pre_train = pd.concat([batch1, batch2, batch3, batch4, batch5, batch6, batch7, batch8, batch9, batch10])
    pre_test = pd.concat([batch11, batch12, batch13, batch14])
    
    del batches, batch1, batch2, batch3, batch4, batch5, batch6, batch7, batch8, batch9, batch10, batch11, batch12, batch13, batch14, train, test, min_max
    
    return pre_train, pre_test


def scale_fillna(pre_train, pre_test):
    '''Fill NA values that were created in the preprocess'''
    features = [col for col in pre_train.columns if col not in ['open_channels', 'set', 'time', 'batch']]
    pre_train = pre_train.replace([np.inf, -np.inf], np.nan)
    pre_test = pre_test.replace([np.inf, -np.inf], np.nan)
    pre_train.fillna(0, inplace = True)
    pre_test.fillna(0, inplace = True)

    return pre_train, pre_test

# feature engineering
pre_train, pre_test = preprocess(train, test)
# reduce memory usage
# scaling and filling missing values
pre_train, pre_test = scale_fillna(pre_train, pre_test)
del train, test
gc.collect()

NameError: name 'train' is not defined

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

def split_data(df, features):
    '''Shuffle the data and split into training and validation data'''
    split1 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=97)
    for train_index, valid_index in split1.split(df, df['open_channels']):
        strat_train_set = df.loc[train_index]
        strat_valid_set = df.loc[valid_index]

    X_train, y_train = strat_train_set[features], strat_train_set['open_channels']
    X_valid, y_valid = strat_valid_set[features], strat_valid_set['open_channels']
    
    return X_train, X_valid, y_train, y_valid

We developed a simple network but taking the input also as an input of the final layers, this way the netwwork learns both deep patterns and simple rules. It was introduced by Heng-Tze Cheng as a Wide and Deep Neural Network.

<img src="resources/image.png">

In [ ]:
def create_model(X_train):
    '''Creates a new model based on the training instance'''
    from keras import backend as K
    from keras import metrics

    K.clear_session()
    neurons = 50

    input_A = keras.layers.Input(shape=X_train.shape[1:])
    hidden1 = keras.layers.Dense(neurons, activation="selu", kernel_initializer="lecun_normal")(input_A)
    hidden2 = keras.layers.Dense(neurons, activation="selu", kernel_initializer="lecun_normal")(hidden1)
    hidden3 = keras.layers.Dense(neurons, activation="selu", kernel_initializer="lecun_normal")(hidden2)
    concat = keras.layers.concatenate([input_A, hidden3])
    output = keras.layers.Dense(11, activation="softmax")(concat)

    model = keras.models.Model(inputs=[input_A], outputs=[output])
    return model

In [ ]:
def train_model(model, X_train, X_valid, y_train, y_valid):
    '''Trains the model using Deep & Wide approach'''
    from sklearn.utils import class_weight
    
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.7, patience=3)

    class_weight_values = class_weight.compute_class_weight('balanced'
                                                   ,np.unique(y_train)
                                                   ,y_train)


    checkpoint_cb = keras.callbacks.ModelCheckpoint("Deep_Wide.h5", save_best_only=True)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True)

    model.compile(loss="sparse_categorical_crossentropy",
                 optimizer="nadam",
                 metrics=['accuracy'])
    
    history = model.fit(X_train, y_train, epochs=30,batch_size=16,class_weight=class_weight_values, 
                       validation_data=(X_valid, y_valid), callbacks=[lr_scheduler, checkpoint_cb, early_stopping_cb])


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

def score(X):
    '''Calculates F1 score on the validation data'''
    Y_pred = model.predict(X).argmax(axis=1)
    score = f1_score(y_valid, Y_pred, average="macro")
    print ('F1 Score (Neural Network): '+str(score))
    return Y_pred

In [ ]:
def save_model(model):
    model.save("models/Deep_Wide.h5")

In [ ]:
from sklearn.preprocessing import StandardScaler

features = [col for col in pre_train.columns if col not in ['open_channels', 'set', 'time', 'batch']] # define the features for training
X_train, X_valid, y_train, y_valid = split_data(pre_train, features)
   
scaler = StandardScaler()
scale = scaler.fit(X_train)
X_train = scale.transform(X_train)
X_valid = scale.transform(X_valid)
X_test = scale.transform(pre_test[features])
                                           
if os.path.exists('/models/Deep_Wide.h5'):
      model = keras.models.load_model('/models/Deep_Wide.h5')
elif os.path.exists('/kaggle/input/models/Deep_Wide.h5'):
    model = keras.models.load_model('/kaggle/input/models/Deep_Wide.h5')
else:
    model = create_model(X_train)
    train_model(model, X_train, X_valid, y_train, y_valid)

save_model(model)
score(X_valid)

y_pred = model.predict(X_test).argmax(axis=1)

In [ ]:
submission.open_channels = y_pred
submission.to_csv("submission.csv",index=False)